# Número óptimo de Tópicos con métrica Coherence

In [1]:
import sys
sys.path.insert(0, '..')
from utils.cargar import df_caso
from utils.preprocesamiento import process_df, StemmerTokenizer
from sklearn.model_selection import train_test_split

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

caso = 'alicia'

df = df_caso(caso)
df = process_df(df,'comment','sel',verbose=True)
df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

80 rows found with non string elements for column comment (0.65%)
Deleting 2326 columns for which max target value is over 7 (18.76%)
9991 available rows after processing


In [2]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

In [3]:
from gensim.models import CoherenceModel

# Set the range of topic numbers to try
min_topics = 2
max_topics = 10
step_size = 1

# Initialize variables for best coherence score and best number of topics
best_coherence_score = -1
best_num_topics = -1

# Iterate over the range of topic numbers
for num_topics in range(min_topics, max_topics+1, step_size):
    # Train the LDA model
    lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=num_topics, id2word=dictionary, passes=10)
    
    # Calculate coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=tokenized_corpus, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    # Check if coherence score is the best so far
    if coherence_score > best_coherence_score:
        best_coherence_score = coherence_score
        best_num_topics = num_topics

# Print the best number of topics
print(f"Best number of topics: {best_num_topics}")

Best number of topics: 6


In [4]:
%%time
lda_model_opt = gensim.models.LdaModel(doc_term_matrix, num_topics=best_num_topics, id2word=dictionary, passes=10)

CPU times: user 25.3 s, sys: 3.95 ms, total: 25.3 s
Wall time: 25.3 s


In [5]:
# Print the generated topics
topics = lda_model_opt.print_topics(num_topics=num_topics)
for topic in topics:
    print(topic)

(0, '0.043*"reputación" + 0.030*"transparencia" + 0.028*"ser" + 0.023*"usuarios" + 0.020*"si" + 0.019*"transparente" + 0.018*"mantener" + 0.015*"mandante" + 0.013*"falencias" + 0.013*"profesional"')
(1, '0.028*"entregar" + 0.025*"producto" + 0.022*"mejor" + 0.019*"calidad" + 0.018*"proyecto" + 0.018*"importante" + 0.017*"tiempo" + 0.015*"plazo" + 0.014*"si" + 0.013*"bien"')
(2, '0.038*"contexto" + 0.030*"acuerdos" + 0.025*"condiciones" + 0.021*"cumplir" + 0.020*"Alicia" + 0.018*"pandemia" + 0.017*"proyecto" + 0.015*"debe" + 0.012*"situación" + 0.012*"mundial"')
(3, '0.018*"debe" + 0.012*"proyecto" + 0.011*"cumplir" + 0.009*"plazos" + 0.008*"importante" + 0.008*"postura" + 0.007*"plazo" + 0.006*"Alicia" + 0.005*"empresa" + 0.005*"bien"')
(4, '0.040*"proyecto" + 0.019*"si" + 0.019*"puede" + 0.016*"trabajo" + 0.014*"bien" + 0.013*"entregar" + 0.011*"ser" + 0.011*"consecuencias" + 0.010*"plazo" + 0.010*"podría"')
(5, '0.066*"proyecto" + 0.037*"criterios" + 0.030*"técnicos" + 0.025*"prioriz

In [6]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model_opt.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Es importante producir producto aceptable incluso si demora pues hacer producto mal puede llevar repercuciones grandes

Topic 0: 0.08216912299394608
Topic 1: 0.2802342474460602
Topic 4: 0.6077150702476501
